In [ ]:
import kinase_library as kl
testsequence='PSVEPPLsQETFSDL'

# Create a Substrate object with a target sequence (example: p53 S33)
s = kl.Substrate(testsequence)  # Lowercase 's' indicates a phosphoserine

# Predict potential kinase interactions for the substrate
s.predict()

In [ ]:
"""Task 1 - to write code mutating first amino acid to every other possible amino acid"""
#how to replace one letter for another in test sequence.

#how to replace one letter for each amino acid



In [ ]:
mutated_sequences = [] #creates a list to store the mutated amino acids
amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # defines the standard 20 amino acids
for i, original_aa in enumerate(testsequence): #indexes the original amino acids 
    for new_aa in amino_acids: #creates a loop of all amino acids
        if new_aa != original_aa:  # Avoid replacing with itself
                mutated_seq = testsequence[:i] + new_aa + testsequence[i+1:] #creates a new sequence (mutated seq)
                mutated_sequences.append(mutated_seq) #stores mutated sequence

print(f'Mutated sequences (# sequences = {len(mutated_sequences)})')
print(mutated_sequences)
 #shows mutations at each position


        

In [ ]:
'''
Task2 - Run these sequences through the predict function and store the output as a csv file. Look at pandas.to_csv() for this.
Output should be 280 separate csv files. Don't worry about a naming convention yet. 
'''



In [ ]:
import os
os.makedirs("mutations", exist_ok=True)
#creates a new folder called mutations to put the sequence predictions into

In [ ]:
import kinase_library as kl
import pandas as pd

testsequence = 'PSVEPPLsQETFSDL'  
amino_acids = "ACDEFGHIKLMNPQRSTVWY"
mutated_sequences = []  
wild_type_filename = f"mutations/wildtype.csv"


# Predict kinase data for the wild-type sequence
try:
    wild_type_substrate = kl.Substrate(testsequence)
    wild_type_predictions = wild_type_substrate.predict()

    # Save wild-type predictions to CSV
    df_wild_type = pd.DataFrame(wild_type_predictions)
    df_wild_type.to_csv(wild_type_filename, index=False)

    print(f"Wild-type sequence predictions saved to {wild_type_filename}")

except Exception as e:
    print(f"Error processing wild-type sequence: {e}")

for i, original_aa in enumerate(testsequence):  
    for new_aa in amino_acids:
        if new_aa != original_aa:  
                mutated_seq = testsequence[:i] + new_aa + testsequence[i+1:] 
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))  


print(f"Total mutations generated: {len(mutated_sequences)}") 

for i, original_aa, new_aa, mutated_seq in mutated_sequences: 
    try:
        s = kl.Substrate(mutated_seq)
        predictions = s.predict()


        df = pd.DataFrame(predictions)

        # Save to CSV file with a structured name
        filename = f"mutations/mutation_Pos{i}_{original_aa}_to_{new_aa}.csv"
        df.to_csv(filename, index=False)

    except Exception as e:
        print(f"Skipping mutation at position {i} ({original_aa} → {new_aa}): {e}")





In [ ]:
'''
Task3 - Run through the 286 csv files and extract the order of the kinases. Then using the test sequence as the reference.
Compute a metric for each pair using something similar to LCS (link for inspo attached - https://chatgpt.com/share/67c705a9-9c74-8001-a8da-eac57f06384f)  
'''

In [ ]:
import os
import pandas as pd
from difflib import SequenceMatcher

#This extracts the kinase orders from 269 files - one being the reference sequence.
#I assume some mutations at position 7 are skipped due to the lack of phosphoacceptor.
#So giving 18 fewer LCS scores compared to the number of CSV files generated earlier.
#This is consistent with the data shown on the heatmap in the next cell.

mut_predictions = "mutations/"

wildtype_path = os.path.join(mut_predictions, "wildtype.csv")

df_wildtype = pd.read_csv(wildtype_path)

wildtype_kinase_order = df_wildtype["Score Rank"].tolist()  #extracts kinase orders from the reference sequence

kinase_orders = {}

#reads all files to extract the kinase orders (except wildtype as that's already been done)
for file in os.listdir(mut_predictions):
    if file.endswith(".csv") and file != "wildtype.csv":
        file_path = os.path.join(mut_predictions, file)
        try:
            df = pd.read_csv(file_path)
            
            if "Score Rank" in df.columns:
                kinase_orders[file] = df["Score Rank"].tolist()  # Store kinase order
            else:
                print(f"Skipping {file}: No 'Score Rank' column found.") 
        
        except Exception as e:
            print(f"Error reading {file}: {e}")   #debugging steps

print(f"Extracted kinase orders from {len(kinase_orders)} mutation CSVs.") 

# Function to compute Longest Common Subsequence (LCS)
def longest_common_subsequence(seq1, seq2):
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

#finds the LCS between wildtype and mut seqs
lcs_scores = {}
for file, order in kinase_orders.items():
    lcs_length = longest_common_subsequence(order, wildtype_kinase_order)
    lcs_scores[file] = lcs_length

#normalizes LCS scores between 0 and 1
if lcs_scores:
    lcs_min = min(lcs_scores.values())
    lcs_max = max(lcs_scores.values())

    normalized_lcs = {file: (score - lcs_min) / (lcs_max - lcs_min) for file, score in lcs_scores.items()}

   
    df_lcs = pd.DataFrame(list(normalized_lcs.items()), columns=['Mutation_File', 'Normalized_LCS'])
    
    print("\nNormalized LCS Scores:")
    print(df_lcs.sort_values(by="Normalized_LCS", ascending=True))  # Sort by lowest LCS

else:
    print("No LCS scores calculated. Ensure kinase_orders contains data.")


In [ ]:
'''
Task 4: generate heatmaps of LCS ratio for each mutation on the y axis and position on the x axis. I like plotly heatmaps for this
https://plotly.com/python/heatmaps/. So output will be a heat map showing hotspots for the mutations that disrup the phosphorylation motif the most
'''

In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np

lcs_data = []

for file, normalized_score in normalized_lcs.items():
    
    match = re.match(r"mutation_Pos(\d+)_\w+_to_(\w+)\.csv", file)

    if match:
        position = int(match.group(1)) 
        mutated_aa = match.group(2)  

        lcs_data.append([position, mutated_aa, normalized_score])


df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])


heatmap_matrix = df_lcs.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio')


fig = px.imshow(
    heatmap_matrix,
    labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
    color_continuous_scale='Temps',
    title="Heatmap of LCS Ratios for Each Mutation",
)

# Format axes
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        dtick=1  # Show every position
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=1  # Show every position
    ),
    coloraxis_colorbar=dict(
        tickvals=[0, 0.25, 0.5, 0.75, 1],  # Set scale from 0 to 1
        ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"]
    ),
    plot_bgcolor='white'  
)
fig.show()






In [ ]:
"Generate tables and subplots showing cases for high and low LCS"

In [ ]:
import os
import pandas as pd
from difflib import SequenceMatcher
import matplotlib.pyplot as plt

# Define paths and variables
mut_predictions = "C:/Users/ricea/OneDrive/Documents/GitHub/Annabel-BSc-Project/mutations"
kinase_orders = {}
mutated_sequences = []

def extract_mutated_sequences():
    """Reads CSV files and extracts mutated sequences from the file names."""
    global mutated_sequences
    for file in os.listdir(mut_predictions):  # Finds the CSVs, reads the CSVs
        if file.endswith(".csv") and file != "wildtype.csv":  # Exclude wildtype.csv
            # Extract position, original amino acid, and new amino acid from the file name
            parts = file.split('_')
            original_aa = parts[2]  # This is the original amino acid
            new_aa = parts[4].split('.')[0]  # This is the new amino acid (without ".csv")
            
            # Store mutated sequence information (e.g., "P_to_A")
            mutated_sequence = f"{original_aa}_to_{new_aa}"
            mutated_sequences.append(mutated_sequence)  # Store mutated sequence

    print(f"Extracted {len(mutated_sequences)} mutated sequences from filenames.")  # Checks if it's done

def extract_kinase_orders():
    """Reads CSV files and extracts kinase orders."""
    global kinase_orders
    for file in os.listdir(mut_predictions):  # Finds the CSVs, reads the CSVs
        if file.endswith(".csv"):
            file_path = os.path.join(mut_predictions, file)
            try:
                df = pd.read_csv(file_path)

                # Check if "Score Rank" column exists
                if "Score Rank" in df.columns:
                    kinase_list = df["Score Rank"].tolist()
                    kinase_orders[file] = kinase_list  # Store kinase order for each mutation
                else:
                    print(f"Skipping {file}: No 'Score Rank' column found.")  # Proofreads

            except Exception as e:
                print(f"Error reading {file}: {e}")  # Proofreads

    print(f"Extracted kinase orders from {len(kinase_orders)} CSV files.")  # Checks if it's done

def longest_common_subsequence(seq1, seq2):
    """Computes the Longest Common Subsequence (LCS) length between two lists."""
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

def calculate_lcs_scores():
    """Calculates the LCS score between mutated sequences and wildtype sequence."""
    pairwise_lcs = {}
    
    # Wildtype sequence from the "wildtype.csv" file
    wildtype_sequence = kinase_orders.get("wildtype.csv", [])
    
    if not wildtype_sequence:
        print("Wildtype sequence not found in the kinase orders.")
        return {}

    for mutated_sequence in mutated_sequences:
        # Look up the kinase order for the mutated sequence file
        mutated_file = f"mutation_{mutated_sequence}.csv"  # Using the mutated sequence format
        mutated_order = next((order for filename, order in kinase_orders.items() if mutated_sequence in filename), None)
        
        if mutated_order:  # If mutated order is found
            lcs_length = longest_common_subsequence(wildtype_sequence, mutated_order)
            pairwise_lcs[mutated_sequence] = lcs_length
        else:
            print(f"Skipping mutated sequence: {mutated_sequence}, order not found.")
    
    # Normalize LCS scores
    lcs_scores = list(pairwise_lcs.values())
    lcs_min = min(lcs_scores)
    lcs_max = max(lcs_scores)

    # Prevent division by zero if all scores are the same
    if lcs_max - lcs_min == 0:
        print("All LCS scores are identical. Normalization skipped.")
        normalized_lcs = {seq: 1.0 for seq in pairwise_lcs}  # Assign all a score of 1
    else:
        normalized_lcs = {seq: (score - lcs_min) / (lcs_max - lcs_min) for seq, score in pairwise_lcs.items()}

    return normalized_lcs

def display_lcs_tables(normalized_lcs):
    """Displays two side-by-side tables for LCS scores in two specific ranges."""
    lcs_groups = {
        '0.8-1.0': [],
        '0.0-0.3': []
    }

    # Group the data by the LCS score ranges
    for mutated_sequence, normalized_score in normalized_lcs.items():
        if 0.8 <= normalized_score <= 1.0:
            lcs_groups['0.8-1.0'].append((mutated_sequence, normalized_score))
        elif 0.0 <= normalized_score <= 0.3:
            lcs_groups['0.0-0.3'].append((mutated_sequence, normalized_score))

    # Create side-by-side tables for each LCS group
    fig, axes = plt.subplots(1, 2, figsize=(18, 5))

    for idx, (score_group, ax) in enumerate(zip(['0.8-1.0', '0.0-0.3'], axes)):
        data = lcs_groups[score_group]

        # Only create a table if there is data
        if data:
            table_data = [(pair[0], f"{pair[1]:.4f}") for pair in data]

            # Create a table
            table = ax.table(cellText=table_data,
                            colLabels=["Mutated Sequence", "Normalized LCS Score"],
                            loc="center",
                            cellLoc="center",
                            colColours=["#f5f5f5"] * 2)

            ax.axis("off")
            ax.set_title(f"LCS Score: {score_group}", fontsize=14)
        else:
            ax.text(0.5, 0.5, f"No data for LCS = {score_group}",
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=12, color='red')

    plt.tight_layout()
    plt.show()

# Example of using the functions:
# Run this once to extract mutated sequences, kinase orders, and calculate LCS scores
extract_mutated_sequences()
extract_kinase_orders()
normalized_lcs = calculate_lcs_scores()

# Then you can run the display function whenever you want to show the tables
if normalized_lcs:
    display_lcs_tables(normalized_lcs)


In [ ]:
'''
Task 5. Using function below. Get a workflow up and running to ping uniprot servers, extract protein sequence 
in the format needed by kinase library to run the code on extracted sequences. You'll need to import the request library using
pip install requests
'''



In [ ]:
import requests

def extract_genomic_information_from_uniprot_id(uniprot_id):
    '''
    Takes in uniprot ID as a string input and pings the Uniprot API to extract genomic coordinates of the protein and exons.
    Metadata such as name, taxID, protein sequence, genome assembly name,  ENSEMBL GeneID, ENSEMBL Transcript ID and ENSEMBL Translations IDs is included alongside the extracted coordinates.  

    Args:
    uniprot_id (str): uniprot ID

    Returns:
    genomic_information (pd.DataFrame): DataFrame containing genomic coordinates of the protein of interest alongside exon positions and metadata 
    '''
    genomic_information = pd.DataFrame()
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/coordinates/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        
        # Check if the request was successful
        response_protein.raise_for_status()
        
        # Load JSON response
        response_protein = response_protein.json()
        
        # Check if response is not empty
        if response_protein:
            response_protein_normalise = pd.json_normalize(
                response_protein, 
                record_path=['gnCoordinate', 'genomicLocation', 'exon'], 
                meta=['accession', 'name', 'taxid', 'sequence', 
                      ['gnCoordinate', 'genomicLocation', 'chromosome'], 
                      ['gnCoordinate', 'genomicLocation', 'start'], 
                      ['gnCoordinate', 'genomicLocation', 'end'], 
                      ['gnCoordinate', 'genomicLocation', 'reverseStrand'], 
                      ['gnCoordinate', 'genomicLocation', 'nucleotideId'], 
                      ['gnCoordinate', 'genomicLocation', 'assemblyName'], 
                      ['gnCoordinate', 'ensemblGeneId'], 
                      ['gnCoordinate', 'ensemblTranscriptId'], 
                      ['gnCoordinate', 'ensemblTranslationId']],
                record_prefix='exon_'
            )

            # Group and aggregate exon information
            response_protein_normalise = response_protein_normalise.groupby([
                'accession', 'name', 'taxid', 'sequence', 
                'gnCoordinate.genomicLocation.chromosome', 
                'gnCoordinate.genomicLocation.start', 
                'gnCoordinate.genomicLocation.end', 
                'gnCoordinate.genomicLocation.reverseStrand', 
                'gnCoordinate.genomicLocation.nucleotideId', 
                'gnCoordinate.genomicLocation.assemblyName', 
                'gnCoordinate.ensemblGeneId', 
                'gnCoordinate.ensemblTranscriptId', 
                'gnCoordinate.ensemblTranslationId'
            ]).agg({
                'exon_id': lambda x: ','.join(map(str, x)),
                'exon_proteinLocation.begin.position': lambda x: ','.join(map(str, x)),                    
                'exon_proteinLocation.end.position': lambda x: ','.join(map(str, x)),
                'exon_genomeLocation.begin.position': lambda x: ','.join(map(str, x)),                    
                'exon_genomeLocation.end.position': lambda x: ','.join(map(str, x))
            }).reset_index()

            # Concatenate to the main DataFrame
            genomic_information = pd.concat([genomic_information, response_protein_normalise], ignore_index=True)
        else:
            print(f"No data found for UniProt ID: {uniprot_id}")
            
    except Exception as e:
        print(f"An error occurred: {e}")

    return genomic_information

In [ ]:
#Example usage
test_df = extract_genomic_information_from_uniprot_id('O15533')
display(test_df)



In [ ]:
import requests
import pandas as pd

uniprot_id = ('O95394-1') #modify these to protein of interest
reference_position = 61

def extract_genomic_information_from_uniprot_id(uniprot_id, reference_position):
    '''
    Extracts genomic coordinates and metadata from UniProt, then retrieves a sequence segment
    surrounding a specified reference position.
    '''
    genomic_information = pd.DataFrame()
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        
        # Check if the request was successful
        response_protein.raise_for_status()
        
        # Load JSON response
        response_protein = response_protein.json()
        
        # Extract protein sequence
        protein_sequence = response_protein.get("sequence", {}).get("sequence", "")
        
        if not protein_sequence:
            print(f"No sequence found for UniProt ID: {uniprot_id}")
            return None
        
        # Extract 7 amino acids upstream and downstream
        start = max(0, reference_position - 7)  # Ensure start is not negative
        end = min(len(protein_sequence), reference_position + 8)  # Ensure end is within bounds
        extracted_sequence = protein_sequence[start:end]

        # Get genomic metadata
        genomic_information = {
            "UniProt_ID": uniprot_id,
            "Protein_Name": response_protein.get("protein", {}).get("recommendedName", {}).get("fullName", ""),
            "TaxID": response_protein.get("organism", {}).get("taxid", ""),
            "Sequence_Length": len(protein_sequence),
            "Reference_Position": reference_position,
            "Extracted_Sequence": extracted_sequence
        }

        return genomic_information

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

result = extract_genomic_information_from_uniprot_id(uniprot_id, reference_position)

if result:
    print(f"Extracted Sequence: {result['Extracted_Sequence']}")


In [ ]:
'This is me working out how to make this code work for a different sequence, ie the extracted sequence. '
'Scroll to the bottom to see this synthesised in one cell with user prompts.'

In [ ]:
def generate_mutated_sequences(seq_of_interest):
    """
    Generates all possible single-point mutations for a given amino acid sequence.

    Args:
        seq_of_interest (str): The original amino acid sequence.

    Returns:
        list: A list containing all possible mutated sequences.
    """
    mutated_sequences = []  # List to store mutated sequences
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # Standard 20 amino acids

    for i, original_aa in enumerate(seq_of_interest):  # Iterate over each amino acid
        for new_aa in amino_acids:  # Try replacing with each amino acid
            if new_aa != original_aa:  # Avoid replacing with itself
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:]
                mutated_sequences.append(mutated_seq)

    return mutated_sequences


seq_of_interest = "STIGVMVTASHNPEE"
mutated_sequences = generate_mutated_sequences(seq_of_interest)

print(f"Mutated sequences (# sequences = {len(mutated_sequences)})")
print(mutated_sequences)  # Shows mutations at each position

        

In [ ]:
import os
os.makedirs("mutations2", exist_ok=True)
#creates a new folder called mutations2 to put the sequence predictions into

In [ ]:
import kinase_library as kl
import pandas as pd

seq_of_interest = 'STIGVMVTASHNPEE'  
amino_acids = "ACDEFGHIKLMNPQRSTVWY"
mutated_sequences = []  
wild_type_filename = f"mutations2/wildtype2.csv"


# Predict kinase data for the wild-type sequence
try:
    wild_type_substrate = kl.Substrate(seq_of_interest)
    wild_type_predictions = wild_type_substrate.predict()

    # Save wild-type predictions to CSV
    df_wild_type = pd.DataFrame(wild_type_predictions)
    df_wild_type.to_csv(wild_type_filename, index=False)

    print(f"Wild-type sequence predictions saved to {wild_type_filename}")

except Exception as e:
    print(f"Error processing wild-type sequence: {e}")

for i, original_aa in enumerate(seq_of_interest):  
    for new_aa in amino_acids:
        if new_aa != original_aa:  
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:] 
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))  


print(f"Total mutations generated: {len(mutated_sequences)}") 

for i, original_aa, new_aa, mutated_seq in mutated_sequences: 
    try:
        s = kl.Substrate(mutated_seq)
        predictions = s.predict()


        df = pd.DataFrame(predictions)

        # Save to CSV file with a structured name
        filename = f"mutations2/mutation_Pos{i}_{original_aa}_to_{new_aa}.csv"
        df.to_csv(filename, index=False)

    except Exception as e:
        print(f"Skipping mutation at position {i} ({original_aa} → {new_aa}): {e}")





In [ ]:
import os
import pandas as pd
from difflib import SequenceMatcher


mut_predictions = "mutations2/"

wildtype_path = os.path.join(mut_predictions, "wildtype2.csv")

df_wildtype = pd.read_csv(wildtype_path)

wildtype_kinase_order = df_wildtype["Score Rank"].tolist()  #extracts kinase orders from the reference sequence

kinase_orders = {}

#reads all files to extract the kinase orders (except wildtype as that's already been done)
for file in os.listdir(mut_predictions):
    if file.endswith(".csv") and file != "wildtype.csv":
        file_path = os.path.join(mut_predictions, file)
        try:
            df = pd.read_csv(file_path)
            
            if "Score Rank" in df.columns:
                kinase_orders[file] = df["Score Rank"].tolist()  # Store kinase order
            else:
                print(f"Skipping {file}: No 'Score Rank' column found.") 
        
        except Exception as e:
            print(f"Error reading {file}: {e}")   #debugging steps

print(f"Extracted kinase orders from {len(kinase_orders)} mutation CSVs.") 

# Function to compute Longest Common Subsequence (LCS)
def longest_common_subsequence(seq1, seq2):
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

#finds the LCS between wildtype and mut seqs
lcs_scores = {}
for file, order in kinase_orders.items():
    lcs_length = longest_common_subsequence(order, wildtype_kinase_order)
    lcs_scores[file] = lcs_length

#normalizes LCS scores between 0 and 1
if lcs_scores:
    lcs_min = min(lcs_scores.values())
    lcs_max = max(lcs_scores.values())

    normalized_lcs = {file: (score - lcs_min) / (lcs_max - lcs_min) for file, score in lcs_scores.items()}

   
    df_lcs = pd.DataFrame(list(normalized_lcs.items()), columns=['Mutation_File', 'Normalized_LCS'])
    
    print("\nNormalized LCS Scores:")
    print(df_lcs.sort_values(by="Normalized_LCS", ascending=True))  # Sort by lowest LCS

else:
    print("No LCS scores calculated. Ensure kinase_orders contains data.")


In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np

lcs_data = []

for file, normalized_score in normalized_lcs.items():
    
    match = re.match(r"mutation_Pos(\d+)_\w+_to_(\w+)\.csv", file)

    if match:
        position = int(match.group(1)) 
        mutated_aa = match.group(2)  

        lcs_data.append([position, mutated_aa, normalized_score])


df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])


heatmap_matrix = df_lcs.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio')


fig = px.imshow(
    heatmap_matrix,
    labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
    color_continuous_scale='Temps',
    title="Heatmap of LCS Ratios for Each Mutation",
)

# Format axes
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        dtick=1  # Show every position
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=1  # Show every position
    ),
    coloraxis_colorbar=dict(
        tickvals=[0, 0.25, 0.5, 0.75, 1],  # Set scale from 0 to 1
        ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"]
    ),
    plot_bgcolor='white'  
)
fig.show()






In [ ]:
'Fully working function with User Prompts to synthesise a heatmap from a given UniProt ID and reference position'

In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np
from difflib import SequenceMatcher
import kinase_library as kl
import requests 

# Function to extract genomic information from UniProt
def extract_genomic_information_from_uniprot_id(uniprot_id, reference_position):
    '''
    Extracts genomic coordinates and metadata from UniProt, then retrieves a sequence segment
    surrounding a specified reference position.
    '''
    genomic_information = pd.DataFrame()
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        
        # Check if the request was successful
        response_protein.raise_for_status()
        
        # Load JSON response
        response_protein = response_protein.json()
        
        # Extract protein sequence
        protein_sequence = response_protein.get("sequence", {}).get("sequence", "")
        
        if not protein_sequence:
            print(f"No sequence found for UniProt ID: {uniprot_id}")
            return None
        
        # Subtract 1 from the reference position to match indexing (UniProt API is 1-based, so this handles it)
        reference_position -= 1  # Convert to 0-based index for Python
        
        # Extract 7 amino acids upstream and downstream
        start = max(0, reference_position - 7)  # Ensure start is not negative
        end = min(len(protein_sequence), reference_position + 8)  # Ensure end is within bounds
        extracted_sequence = protein_sequence[start:end]

        # Get genomic metadata
        genomic_information = {
            "UniProt_ID": uniprot_id,
            "Protein_Name": response_protein.get("protein", {}).get("recommendedName", {}).get("fullName", ""),
            "TaxID": response_protein.get("organism", {}).get("taxid", ""),
            "Sequence_Length": len(protein_sequence),
            "Reference_Position": reference_position + 1,  # Return to 1-based indexing for user
            "Extracted_Sequence": extracted_sequence
        }

        return genomic_information

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to generate all mutated sequences
def generate_mutated_sequences(seq_of_interest):
    """
    Generates all possible single-point mutations for a given amino acid sequence.

    Args:
        seq_of_interest (str): The original amino acid sequence.

    Returns:
        list: A list containing all possible mutated sequences.
    """
    mutated_sequences = []  # List to store mutated sequences
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # Standard 20 amino acids

    for i, original_aa in enumerate(seq_of_interest):  # Iterate over each amino acid
        for new_aa in amino_acids:  # Try replacing with each amino acid
            if new_aa != original_aa:  # Avoid replacing with itself
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:]
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))

    return mutated_sequences

# User input for UniProt ID and reference position
uniprot_id = input("Enter the UniProt ID: ")
reference_position = int(input("Enter the reference position: "))

# Extract sequence and genomic data
result = extract_genomic_information_from_uniprot_id(uniprot_id, reference_position)

if result:
    print(f"Extracted Sequence: {result['Extracted_Sequence']}")  # Output the extracted sequence

    # Mutate the sequence of interest
    mutated_sequences = generate_mutated_sequences(result['Extracted_Sequence'])

    print(f"Mutated sequences (# sequences = {len(mutated_sequences)})")
    print(mutated_sequences)  # Shows mutations at each position

    # Set up directory for mutation results
    os.makedirs("mutations2", exist_ok=True)

    # Wild-type predictions
    try:
        wild_type_substrate = kl.Substrate(result['Extracted_Sequence'])
        wild_type_predictions = wild_type_substrate.predict()

        wild_type_filename = f"mutations2/wildtype2.csv"
        df_wild_type = pd.DataFrame(wild_type_predictions)
        df_wild_type.to_csv(wild_type_filename, index=False)

        print(f"Wild-type sequence predictions saved to {wild_type_filename}")

    except Exception as e:
        print(f"Error processing wild-type sequence: {e}")

    # Predicting mutated sequences and saving
    for i, original_aa, new_aa, mutated_seq in mutated_sequences:  
        try:
            s = kl.Substrate(mutated_seq)
            predictions = s.predict()

            df = pd.DataFrame(predictions)

            # Save to CSV file with a structured name (corrected format)
            filename = f"mutations2/mutation_Pos{i+1}_{original_aa}_to_{new_aa}.csv"  # Corrected nomenclature
            df.to_csv(filename, index=False)

        except Exception as e:
            print(f"Skipping mutation at position {i+1} ({original_aa} → {new_aa}): {e}")

    # Extract kinase orders from wild-type
    wildtype_path = os.path.join("mutations2", "wildtype2.csv")
    df_wildtype = pd.read_csv(wildtype_path)
    wildtype_kinase_order = df_wildtype["Score Rank"].tolist()

    kinase_orders = {}
    for file in os.listdir("mutations2"):
        if file.endswith(".csv") and file != "wildtype2.csv":
            file_path = os.path.join("mutations2", file)
            try:
                df = pd.read_csv(file_path)

                if "Score Rank" in df.columns:
                    kinase_orders[file] = df["Score Rank"].tolist()
                else:
                    print(f"Skipping {file}: No 'Score Rank' column found.")
            except Exception as e:
                print(f"Error reading {file}: {e}")

    print(f"Extracted kinase orders from {len(kinase_orders)} mutation CSVs.")

    # Function to compute Longest Common Subsequence (LCS)
    def longest_common_subsequence(seq1, seq2):
        matcher = SequenceMatcher(None, seq1, seq2)
        return sum(block.size for block in matcher.get_matching_blocks())

    # Find LCS between wildtype and mutated sequences
    lcs_scores = {}
    for file, order in kinase_orders.items():
        lcs_length = longest_common_subsequence(order, wildtype_kinase_order)
        lcs_scores[file] = lcs_length

    # Normalize LCS scores
    if lcs_scores:
        normalized_lcs = {file: (score - 1) / (310 - 1) for file, score in lcs_scores.items()}


        df_lcs = pd.DataFrame(list(normalized_lcs.items()), columns=['Mutation_File', 'Normalized_LCS'])

        print("\nNormalized LCS Scores:")
        print(df_lcs.sort_values(by="Normalized_LCS", ascending=True))

    # Prepare heatmap of LCS ratios
    lcs_data = []
    for file, normalized_score in normalized_lcs.items():
        match = re.match(r"mutation_Pos(\d+)_\w+_to_(\w+)\.csv", file)

        if match:
            position = int(match.group(1))  


            mutated_aa = match.group(2)

            lcs_data.append([position, mutated_aa, normalized_score])

    df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])

    # Create heatmap matrix
    heatmap_matrix = df_lcs.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)

    # Generate heatmap with Plotly
    fig = px.imshow(
        heatmap_matrix,
        labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
        color_continuous_scale='Temps',
        title="Heatmap of LCS Ratios for Each Mutation",
    )

    # Create a mapping for the positions
position_labels = list(range(-7, 8))  # This creates the range from -7 to +7

# Adjust xaxis tickvals and ticktext
fig.update_layout(
    xaxis=dict(
        tickmode='array', 
        tickvals=list(range(1, 16)),  # Use 1 to 15 for the actual mutation positions
        ticktext=position_labels  # Use the custom tick labels (-7 to +7)
    ),
    yaxis=dict(
        tickmode='linear',
        dtick=1  # Show every position (for y-axis if needed)
    ),
    coloraxis_colorbar=dict(
        tickvals=[0, 0.25, 0.5, 0.75, 1],  # Color bar ticks from 0 to 1
        ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"],
        tickmode='array',  # Ensure it's using the array for ticks
        title="LCS Ratio",  # Optional: add a title for the color bar
        tickangle=0  # Keep tick labels horizontal
    ),
    plot_bgcolor='white'  # Background color (optional)
)

# Show the plot
fig.show()



In [ ]:
'Split the code into functional blocks to facilitate ease of running and storytelling'

In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np
from difflib import SequenceMatcher
import kinase_library as kl
import requests 

# Function to extract genomic information from UniProt
def extract_genomic_information_from_uniprot_id(uniprot_id, reference_position):
    '''
    Extracts genomic coordinates and metadata from UniProt, then retrieves a sequence segment
    surrounding a specified reference position.
    '''
    genomic_information = pd.DataFrame()
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        
        # Check if the request was successful
        response_protein.raise_for_status()
        
        # Load JSON response
        response_protein = response_protein.json()
        
        # Extract protein sequence
        protein_sequence = response_protein.get("sequence", {}).get("sequence", "")
        
        if not protein_sequence:
            print(f"No sequence found for UniProt ID: {uniprot_id}")
            return None
        
        # Subtract 1 from the reference position to match indexing (UniProt API is 1-based, so this handles it)
        reference_position -= 1  # Convert to 0-based index for Python
        
        # Extract 7 amino acids upstream and downstream
        start = max(0, reference_position - 7)  # Ensure start is not negative
        end = min(len(protein_sequence), reference_position + 8)  # Ensure end is within bounds
        extracted_sequence = protein_sequence[start:end]

        # Get genomic metadata
        genomic_information = {
            "UniProt_ID": uniprot_id,
            "Protein_Name": response_protein.get("protein", {}).get("recommendedName", {}).get("fullName", ""),
            "TaxID": response_protein.get("organism", {}).get("taxid", ""),
            "Sequence_Length": len(protein_sequence),
            "Reference_Position": reference_position + 1,  # Return to 1-based indexing for user
            "Extracted_Sequence": extracted_sequence
        }

        return genomic_information

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to generate all mutated sequences
def generate_mutated_sequences(seq_of_interest):
    """
    Generates all possible single-point mutations for a given amino acid sequence.

    Args:
        seq_of_interest (str): The original amino acid sequence.

    Returns:
        list: A list containing all possible mutated sequences.
    """
    mutated_sequences = []  # List to store mutated sequences
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"  # Standard 20 amino acids

    for i, original_aa in enumerate(seq_of_interest):  # Iterate over each amino acid
        for new_aa in amino_acids:  # Try replacing with each amino acid
            if new_aa != original_aa:  # Avoid replacing with itself
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:]
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))

    return mutated_sequences

# Function to save wild-type predictions
def save_wildtype_predictions(uniprot_id, reference_position, extracted_sequence):
    try:
        wild_type_substrate = kl.Substrate(extracted_sequence)
        wild_type_predictions = wild_type_substrate.predict()

        wild_type_filename = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
        df_wild_type = pd.DataFrame(wild_type_predictions)
        df_wild_type.to_csv(wild_type_filename, index=False)

        print(f"Wild-type sequence predictions saved to {wild_type_filename}")

    except Exception as e:
        print(f"Error processing wild-type sequence: {e}")

# Function to save mutated sequences
def save_mutated_predictions(mutated_sequences, uniprot_id, reference_position):
    for i, original_aa, new_aa, mutated_seq in mutated_sequences:
        try:
            s = kl.Substrate(mutated_seq)
            predictions = s.predict()

            df = pd.DataFrame(predictions)

            # Save to CSV file with the updated naming convention
            filename = f"mutations2/{uniprot_id}_{reference_position}_mutation_pos{i+1}_{original_aa}_to_{new_aa}.csv"
            df.to_csv(filename, index=False)

        except Exception as e:
            print(f"Skipping mutation at position {i+1} ({original_aa} → {new_aa}): {e}")

# Function to extract kinase orders from CSV files
def extract_kinase_orders():
    kinase_orders = {}
    for file in os.listdir("mutations2"):
        if file.endswith(".csv") and "wildtype" not in file:
            file_path = os.path.join("mutations2", file)
            try:
                df = pd.read_csv(file_path)

                if "Score Rank" in df.columns:
                    kinase_orders[file] = df["Score Rank"].tolist()
                else:
                    print(f"Skipping {file}: No 'Score Rank' column found.")
            except Exception as e:
                print(f"Error reading {file}: {e}")

    return kinase_orders

# Function to compute Longest Common Subsequence (LCS)
def longest_common_subsequence(seq1, seq2):
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

# Function to normalize LCS scores
def normalize_lcs_scores(lcs_scores):
    return {file: (score - 1) / (310 - 1) for file, score in lcs_scores.items()}

# Function to process LCS data for heatmap and save it
def process_lcs_for_heatmap(normalized_lcs, uniprot_id):
    lcs_data = []
    for file, normalized_score in normalized_lcs.items():
        match = re.match(r"mutation_Pos(\d+)_\w+_to_(\w+)\.csv", file)

        if match:
            position = int(match.group(1))  # Get mutation position
            mutated_aa = match.group(2)  # Get mutated amino acid

            lcs_data.append([position, mutated_aa, normalized_score])

    df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])

    # Save LCS data for later use
    heatmap_filename = f"mutations2/{uniprot_id}_heatmap_data.csv"
    df_lcs.to_csv(heatmap_filename, index=False)
    print(f"LCS data saved to {heatmap_filename}")

# Function to render heatmap from processed data
def render_heatmap_from_data(heatmap_data):
    # Generate heatmap matrix
    heatmap_matrix = heatmap_data.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)

    # Generate heatmap with Plotly
    fig = px.imshow(
        heatmap_matrix,
        labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
        color_continuous_scale='Temps',
        title="Heatmap of LCS Ratios for Each Mutation",
    )

    position_labels = list(range(-7, 8))  # This creates the range from -7 to +7

    fig.update_layout(
        xaxis=dict(
            tickmode='array', 
            tickvals=list(range(1, 16)),  # Use 1 to 15 for the actual mutation positions
            ticktext=position_labels  # Use the custom tick labels (-7 to +7)
        ),
        yaxis=dict(
            tickmode='linear',
            dtick=1  # Show every position (for y-axis if needed)
        ),
        coloraxis_colorbar=dict(
            tickvals=["0, 0.25, 0.5, 0.75, 1"],  # Color bar ticks from 0 to 1
            ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"],
            tickmode='array',  # Ensure it's using the array for ticks
            title="LCS Ratio",  # Optional: add a title for the color bar
            tickangle=0  # Keep tick labels horizontal
        ),
        plot_bgcolor='white'  # Background color 
    )

    # Show the plot
    fig.show()

def run_pipeline():
    # User input for UniProt ID and reference position
    uniprot_id = input("Enter the UniProt ID: ")
    reference_position = int(input("Enter the reference position: "))

    # Extract sequence and genomic data
    result = extract_genomic_information_from_uniprot_id(uniprot_id, reference_position)

    if result:
        print(f"Extracted Sequence: {result['Extracted_Sequence']}")  # Output the extracted sequence

        # Mutate the sequence of interest
        mutated_sequences = generate_mutated_sequences(result['Extracted_Sequence'])

        print(f"Mutated sequences (# sequences = {len(mutated_sequences)})")
        print(mutated_sequences)  # Shows mutations at each position

        # Save wild-type predictions
        save_wildtype_predictions(uniprot_id, reference_position, result['Extracted_Sequence'])

        # Save mutated sequence predictions
        save_mutated_predictions(mutated_sequences, uniprot_id, reference_position)

        # Extract kinase orders from mutated CSV files
        kinase_orders = extract_kinase_orders()

        # Extract kinase order from wildtype
        wildtype_path = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
        df_wildtype = pd.read_csv(wildtype_path)
        wildtype_kinase_order = df_wildtype["Score Rank"].tolist()

        # Compute LCS scores between wildtype and mutated sequences
        lcs_scores = {}
        for file, order in kinase_orders.items():
            lcs_length = longest_common_subsequence(order, wildtype_kinase_order)
            lcs_scores[file] = lcs_length

        # Normalize LCS scores
        normalized_lcs = normalize_lcs_scores(lcs_scores)

        # Process the LCS data for heatmap and save it to a file
        process_lcs_for_heatmap(normalized_lcs, uniprot_id)

        # Now, load the processed LCS data for rendering the heatmap
        heatmap_data = pd.read_csv(f"mutations2/{uniprot_id}_heatmap_data.csv")

        # Render the heatmap using the processed data
        render_heatmap_from_data(heatmap_data)

if __name__ == "__main__":
    run_pipeline()


In [ ]:
'Losing kinase names when saving predictions'
'Regex does not extract mutations and positions correctly'
##Debugging##

In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np
from difflib import SequenceMatcher
import kinase_library as kl
import requests 

# Function to extract genomic information from UniProt
def extract_genomic_information_from_uniprot_id(uniprot_id, reference_position):
    '''
    Extracts genomic coordinates and metadata from UniProt, then retrieves a sequence segment
    surrounding a specified reference position.
    '''
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        
        # Check if the request was successful
        response_protein.raise_for_status()
        
        # Load JSON response
        response_protein = response_protein.json()
        
        # Extract protein sequence
        protein_sequence = response_protein.get("sequence", {}).get("sequence", "")
        
        if not protein_sequence:
            print(f"No sequence found for UniProt ID: {uniprot_id}")
            return None
        
        reference_position -= 1  # Convert to 0-based index for Python
        
        start = max(0, reference_position - 7)
        end = min(len(protein_sequence), reference_position + 8)
        extracted_sequence = protein_sequence[start:end]

        genomic_information = {
            "UniProt_ID": uniprot_id,
            "Protein_Name": response_protein.get("protein", {}).get("recommendedName", {}).get("fullName", ""),
            "TaxID": response_protein.get("organism", {}).get("taxid", ""),
            "Sequence_Length": len(protein_sequence),
            "Reference_Position": reference_position + 1,
            "Extracted_Sequence": extracted_sequence
        }

        return genomic_information

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to generate all mutated sequences
def generate_mutated_sequences(seq_of_interest):
    mutated_sequences = []
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    for i, original_aa in enumerate(seq_of_interest):
        for new_aa in amino_acids:
            if new_aa != original_aa:
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:]
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))

    return mutated_sequences

# Function to save wild-type predictions
def save_wildtype_predictions(uniprot_id, reference_position, extracted_sequence):
    try:
        wild_type_substrate = kl.Substrate(extracted_sequence)
        wild_type_predictions = wild_type_substrate.predict()

        wild_type_filename = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
        df_wild_type = pd.DataFrame(wild_type_predictions).reset_index()
        df_wild_type.to_csv(wild_type_filename, index=False)

        print(f"Wild-type sequence predictions saved to {wild_type_filename}")

    except Exception as e:
        print(f"Error processing wild-type sequence: {e}")

# Function to save mutated sequences
def save_mutated_predictions(mutated_sequences, uniprot_id, reference_position):
    for i, original_aa, new_aa, mutated_seq in mutated_sequences:
        try:
            s = kl.Substrate(mutated_seq)
            predictions = s.predict()

            df = pd.DataFrame(predictions).reset_index()

            filename = f"mutations2/{uniprot_id}_{reference_position}_mutation_pos{i+1}_{original_aa}_to_{new_aa}.csv"
            df.to_csv(filename, index=False)

        except Exception as e:
            print(f"Skipping mutation at position {i+1} ({original_aa} → {new_aa}): {e}")

# Function to extract kinase orders from CSV files
def extract_kinase_orders():
    kinase_orders = {}
    for file in os.listdir("mutations2"):
        if file.endswith(".csv") and "wildtype" not in file:
            file_path = os.path.join("mutations2", file)
            try:
                df = pd.read_csv(file_path)

                if "index" in df.columns:
                    kinase_orders[file] = df["index"].tolist()
                else:
                    print(f"Skipping {file}: No 'index' column found.")
            except Exception as e:
                print(f"Error reading {file}: {e}")

    return kinase_orders

# Function to compute Longest Common Subsequence (LCS)
def longest_common_subsequence(seq1, seq2):
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

# Function to normalize LCS scores
def normalize_lcs_scores(lcs_scores):
    return {file: (score - 1) / (310 - 1) for file, score in lcs_scores.items()}

# Function to process LCS data for heatmap and save it
def process_lcs_for_heatmap(normalized_lcs, uniprot_id, reference_position):
    lcs_data = []
    for file, normalized_score in normalized_lcs.items():
        match = re.match(rf"{uniprot_id}_{reference_position}_mutation_pos(\d+)_(\w+)_to_(\w+)\.csv", file)

        if match:
            position = int(match.group(1))
            mutated_aa = match.group(3)

            lcs_data.append([position, mutated_aa, normalized_score])

    df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])

    heatmap_filename = f"mutations2/{uniprot_id}_{reference_position}_heatmap_data.csv"
    df_lcs.to_csv(heatmap_filename, index=False)
    print(f"LCS data saved to {heatmap_filename}")

# Function to render heatmap from processed data
def render_heatmap_from_data(heatmap_data):
    heatmap_matrix = heatmap_data.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)

    fig = px.imshow(
        heatmap_matrix,
        labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
        color_continuous_scale='Temps',
        title="Heatmap of LCS Ratios for Each Mutation",
    )

    position_labels = list(range(-7, 8))

    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=list(range(1, 16)), ticktext=position_labels),
        yaxis=dict(tickmode='linear', dtick=1),
        coloraxis_colorbar=dict(
            tickvals=[0, 0.25, 0.5, 0.75, 1],
            ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"],
            tickmode='array',
            title="LCS Ratio",
            tickangle=0
        ),
        plot_bgcolor='white'
    )

    fig.show()

# Main pipeline function
def run_pipeline():
    uniprot_id = input("Enter the UniProt ID: ")
    reference_position = int(input("Enter the reference position: "))

    result = extract_genomic_information_from_uniprot_id(uniprot_id, reference_position)

    if result:
        mutated_sequences = generate_mutated_sequences(result['Extracted_Sequence'])

        save_wildtype_predictions(uniprot_id, reference_position, result['Extracted_Sequence'])
        save_mutated_predictions(mutated_sequences, uniprot_id, reference_position)

        kinase_orders = extract_kinase_orders()

        wildtype_path = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
        df_wildtype = pd.read_csv(wildtype_path)

        if "index" not in df_wildtype.columns:
            print(f"Error: 'index' column not found in {wildtype_path}")
            return

        wildtype_kinase_order = df_wildtype["index"].tolist()

        lcs_scores = {file: longest_common_subsequence(order, wildtype_kinase_order) for file, order in kinase_orders.items()}
        normalized_lcs = normalize_lcs_scores(lcs_scores)

        process_lcs_for_heatmap(normalized_lcs, uniprot_id, reference_position)
        render_heatmap_from_data(pd.read_csv(f"mutations2/{uniprot_id}_{reference_position}_heatmap_data.csv"))

if __name__ == "__main__":
    run_pipeline()


In [ ]:
'Modified for batch input of UniProt ID and Reference Position from Excel File'
#Excel file should be named 'input.xlsx' and saved in same directory as the repository
#Column headers MUST be 'uniprot_ID' and 'amino_acid_position' (case-sensitive)

In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np
from difflib import SequenceMatcher
import kinase_library as kl
import requests 

# Function to extract genomic information from UniProt
def extract_genomic_information_from_uniprot_id(uniprot_id, reference_position):
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        response_protein.raise_for_status()
        response_protein = response_protein.json()

        protein_sequence = response_protein.get("sequence", {}).get("sequence", "")
        if not protein_sequence:
            print(f"No sequence found for UniProt ID: {uniprot_id}")
            return None

        reference_position -= 1  # Convert to 0-based index for Python
        start = max(0, reference_position - 7)
        end = min(len(protein_sequence), reference_position + 8)
        extracted_sequence = protein_sequence[start:end]

        return {
            "UniProt_ID": uniprot_id,
            "Protein_Name": response_protein.get("protein", {}).get("recommendedName", {}).get("fullName", ""),
            "TaxID": response_protein.get("organism", {}).get("taxid", ""),
            "Sequence_Length": len(protein_sequence),
            "Reference_Position": reference_position + 1,
            "Extracted_Sequence": extracted_sequence
        }

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to generate all mutated sequences
def generate_mutated_sequences(seq_of_interest):
    mutated_sequences = []
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    for i, original_aa in enumerate(seq_of_interest):
        for new_aa in amino_acids:
            if new_aa != original_aa:
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:]
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))

    return mutated_sequences

# Function to save wild-type predictions
def save_wildtype_predictions(uniprot_id, reference_position, extracted_sequence):
    try:
        wild_type_substrate = kl.Substrate(extracted_sequence)
        wild_type_predictions = wild_type_substrate.predict()
        wild_type_filename = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
        pd.DataFrame(wild_type_predictions).reset_index().to_csv(wild_type_filename, index=False)
        print(f"Wild-type sequence predictions saved to {wild_type_filename}")
    except Exception as e:
        print(f"Error processing wild-type sequence: {e}")

# Function to save mutated sequences
def save_mutated_predictions(mutated_sequences, uniprot_id, reference_position):
    for i, original_aa, new_aa, mutated_seq in mutated_sequences:
        try:
            s = kl.Substrate(mutated_seq)
            predictions = s.predict()
            filename = f"mutations2/{uniprot_id}_{reference_position}_mutation_pos{i+1}_{original_aa}_to_{new_aa}.csv"
            pd.DataFrame(predictions).reset_index().to_csv(filename, index=False)
        except Exception as e:
            print(f"Skipping mutation at position {i+1} ({original_aa} → {new_aa}): {e}")

# Function to extract kinase orders from CSV files
def extract_kinase_orders():
    kinase_orders = {}
    for file in os.listdir("mutations2"):
        if file.endswith(".csv") and "wildtype" not in file:
            file_path = os.path.join("mutations2", file)
            try:
                df = pd.read_csv(file_path)

                # Check for the 'index' column first, then 'position' if 'index' is missing
                if "index" in df.columns:
                    kinase_orders[file] = df["index"].tolist()  # Use 'index' column for kinase order
                elif "Position" in df.columns:
                    kinase_orders[file] = df["Position"].tolist()  # Use 'position' column if 'index' is missing
                else:
                    print(f"Skipping {file}: No 'index' or 'position' column found.")
            except Exception as e:
                print(f"Error reading {file}: {e}")

    return kinase_orders


# Function to compute Longest Common Subsequence (LCS)
def longest_common_subsequence(seq1, seq2):
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

# Function to normalize LCS scores
def normalize_lcs_scores(lcs_scores):
    return {file: (score - 1) / (310 - 1) for file, score in lcs_scores.items()}

# Function to process LCS data for heatmap and save it
def process_lcs_for_heatmap(normalized_lcs, uniprot_id, reference_position):
    lcs_data = []
    for file, normalized_score in normalized_lcs.items():
        match = re.match(rf"{uniprot_id}_{reference_position}_mutation_pos(\d+)_(\w+)_to_(\w+)\.csv", file)
        if match:
            position = int(match.group(1))
            mutated_aa = match.group(3)
            lcs_data.append([position, mutated_aa, normalized_score])

    df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])
    heatmap_filename = f"mutations2/{uniprot_id}_{reference_position}_heatmap_data.csv"
    df_lcs.to_csv(heatmap_filename, index=False)
    print(f"LCS data saved to {heatmap_filename}")

import plotly.express as px
import numpy as np

# Function to render heatmap from processed data
def render_heatmap_from_data(heatmap_data):
    heatmap_matrix = heatmap_data.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)

    fig = px.imshow(
        heatmap_matrix,
        labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
        color_continuous_scale='Temps',
        title="Heatmap of LCS Ratios for Each Mutation",
    )

    position_labels = list(range(-7, 8))

    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=list(range(1, 16)), ticktext=position_labels),
        yaxis=dict(tickmode='linear', dtick=1),
        coloraxis_colorbar=dict(
            tickvals=[0, 0.25, 0.5, 0.75, 1],
            ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"],
            tickmode='array',
            title="LCS Ratio",
            tickangle=0
        ),
        plot_bgcolor='white'
    )

    fig.show()


import pandas as pd

# Main pipeline function
def run_pipeline_from_excel(input_file):
    # Load the input Excel file
    df = pd.read_excel(input_file)

    # Iterate over each row in the Excel file
    for index, row in df.iterrows():
        # Extract the UniProt ID and reference position from the row
        uniprot_id = row["uniprot_ID"]
        reference_position = int(row["amino_acid_position"])

        print(f"Processing {uniprot_id} at position {reference_position}...")

        # Extract genomic information (you need to implement extract_genomic_information_from_uniprot_id)
        result = extract_genomic_information_from_uniprot_id(uniprot_id, reference_position)

        if result:
            # Generate mutated sequences (you need to implement generate_mutated_sequences)
            mutated_sequences = generate_mutated_sequences(result['Extracted_Sequence'])

            # Save wildtype and mutated predictions (you need to implement save_wildtype_predictions and save_mutated_predictions)
            save_wildtype_predictions(uniprot_id, reference_position, result['Extracted_Sequence'])
            save_mutated_predictions(mutated_sequences, uniprot_id, reference_position)

            # Extract kinase orders (you need to implement extract_kinase_orders)
            kinase_orders = extract_kinase_orders()

            # Load the wildtype CSV file
            wildtype_path = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
            try:
                df_wildtype = pd.read_csv(wildtype_path)

                # Check if either "index" or "position" columns are present
                if "index" not in df_wildtype.columns and "position" not in df_wildtype.columns:
                    print(f"Error: Neither 'index' nor 'position' column found in {wildtype_path}")
                    continue  # Skip this entry and go to the next one

                # Use 'index' or 'position' for kinase order
                kinase_order_column = "index" if "index" in df_wildtype.columns else "position"
                wildtype_kinase_order = df_wildtype[kinase_order_column].tolist()

                # Calculate LCS scores
                lcs_scores = {file: longest_common_subsequence(order, wildtype_kinase_order) for file, order in kinase_orders.items()}

                # Normalize LCS scores (you need to implement normalize_lcs_scores)
                normalized_lcs = normalize_lcs_scores(lcs_scores)

                # Process LCS for heatmap and save it (you need to implement process_lcs_for_heatmap and render_heatmap_from_data)
                process_lcs_for_heatmap(normalized_lcs, uniprot_id, reference_position)
                render_heatmap_from_data(pd.read_csv(f"mutations2/{uniprot_id}_{reference_position}_heatmap_data.csv"))

            except FileNotFoundError:
                print(f"Warning: Wildtype file {wildtype_path} not found. Skipping {uniprot_id}.")
                continue  # Skip if the wildtype file is not found

        else:
            print(f"Error: No genomic information found for {uniprot_id} at position {reference_position}. Skipping.")
            continue  # Skip to the next UniProt ID if no genomic information is found


if __name__ == "__main__":
    run_pipeline_from_excel("input.xlsx")


In [ ]:
'Comparison of two heat maps by position - not pairwise, just like for like positions'

In [ ]:
import os
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

def compare_heatmaps(uniprot_id1, uniprot_id2, reference_position1, reference_position2, directory):
    # Construct file paths based on naming convention
    heatmap_file1 = os.path.join(directory, f"{uniprot_id1}_{reference_position1}_heatmap_data.csv")
    heatmap_file2 = os.path.join(directory, f"{uniprot_id2}_{reference_position2}_heatmap_data.csv")
    
    print(f"Looking for files: {heatmap_file1} and {heatmap_file2}")
    
    # Check if both heatmap files exist
    if not os.path.exists(heatmap_file1) or not os.path.exists(heatmap_file2):
        print(f"Error: One or both heatmap files for {uniprot_id1}_{reference_position1} or {uniprot_id2}_{reference_position2} do not exist.")
        return
    
    # Load heatmap data
    df1 = pd.read_csv(heatmap_file1)
    df2 = pd.read_csv(heatmap_file2)
    
    # Ensure that both dataframes have the same structure
    if not {'Mutation', 'Position', 'LCS_Ratio'}.issubset(df1.columns) or not {'Mutation', 'Position', 'LCS_Ratio'}.issubset(df2.columns):
        print(f"Error: Missing required columns in heatmap data files.")
        return
    
    # Pivot the data to create a matrix for each heatmap
    heatmap_matrix1 = df1.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)
    heatmap_matrix2 = df2.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)
    
    # Ensure both matrices have the same positions
    common_positions = heatmap_matrix1.columns.intersection(heatmap_matrix2.columns)
    
    # Perform t-tests for each common position
    p_values = {}
    for position in common_positions:
        data1 = heatmap_matrix1[position].dropna()
        data2 = heatmap_matrix2[position].dropna()
        
        # Perform t-test only if both groups have data
        if len(data1) > 0 and len(data2) > 0:
            t_stat, p_val = ttest_ind(data1, data2)
            p_values[position] = p_val
    
    # Save p-values to a CSV file using the desired naming convention
    output_file = os.path.join(directory, f"{uniprot_id1}_{reference_position1}_vs_{uniprot_id2}_{reference_position2}_p_values.csv")
    p_values_df = pd.DataFrame(list(p_values.items()), columns=['Position', 'P-Value'])
    p_values_df.to_csv(output_file, index=False)
    
    print(f"P-values comparison saved to {output_file}")
    
    # Print positions with p-value smaller than 0.05
    significant_positions = p_values_df[p_values_df['P-Value'] < 0.05]
    print("Significant Positions (p-value < 0.05):")
    print(significant_positions)


uniprot_id1 = "P40337-1"
uniprot_id2 = "P07954-1"
reference_position1 = 72
reference_position2 = 365
directory = "mutations2"

compare_heatmaps(uniprot_id1, uniprot_id2, reference_position1, reference_position2, directory)


In [ ]:
'Pairwise T tests for every position between 2 uniprot IDs'

In [ ]:
import os
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

def compare_heatmaps(uniprot_id1, uniprot_id2, reference_position1, reference_position2, directory):
    # Construct file paths based on naming convention
    heatmap_file1 = os.path.join(directory, f"{uniprot_id1}_{reference_position1}_heatmap_data.csv")
    heatmap_file2 = os.path.join(directory, f"{uniprot_id2}_{reference_position2}_heatmap_data.csv")
    
    print(f"Looking for files: {heatmap_file1} and {heatmap_file2}")
    
    # Check if both heatmap files exist
    if not os.path.exists(heatmap_file1) or not os.path.exists(heatmap_file2):
        print(f"Error: One or both heatmap files for {uniprot_id1}_{reference_position1} or {uniprot_id2}_{reference_position2} do not exist.")
        return
    
    # Load heatmap data
    df1 = pd.read_csv(heatmap_file1)
    df2 = pd.read_csv(heatmap_file2)
    
    # Ensure that both dataframes have the same structure
    if not {'Mutation', 'Position', 'LCS_Ratio'}.issubset(df1.columns) or not {'Mutation', 'Position', 'LCS_Ratio'}.issubset(df2.columns):
        print(f"Error: Missing required columns in heatmap data files.")
        return
    
    # Pivot the data to create a matrix for each heatmap
    heatmap_matrix1 = df1.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)
    heatmap_matrix2 = df2.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)
    
    # Perform t-tests for every pair of positions between both heatmaps
    p_values = []
    for pos1 in heatmap_matrix1.columns:
        for pos2 in heatmap_matrix2.columns:
            data1 = heatmap_matrix1[pos1].dropna()
            data2 = heatmap_matrix2[pos2].dropna()
            
            # Perform t-test only if both groups have data
            if len(data1) > 0 and len(data2) > 0:
                t_stat, p_val = ttest_ind(data1, data2)
                p_values.append((pos1, pos2, p_val))
    
    # Convert the list of p-values into a DataFrame
    p_values_df = pd.DataFrame(p_values, columns=['Position1', 'Position2', 'P-Value'])
    
    # Save the p-values to a CSV file using the desired naming convention (with "pairwise" at the front)
    output_file = os.path.join(directory, f"pairwise_{uniprot_id1}_{reference_position1}_vs_{uniprot_id2}_{reference_position2}_p_values.csv")
    
    # Save all p-values (including those > 0.05)
    p_values_df.to_csv(output_file, index=False)
    
    print(f"P-values comparison saved to {output_file}")
    
    # Filter positions with p-values smaller than 0.05 and print them
    significant_positions = p_values_df[p_values_df['P-Value'] < 0.05]
    print("Significant Positions (p-value < 0.05):")
    print(significant_positions)

uniprot_id1 = "O95394-1"
uniprot_id2 = "P40337-1"
reference_position1 = 62
reference_position2 = 72
directory = "mutations2"

compare_heatmaps(uniprot_id1, uniprot_id2, reference_position1, reference_position2, directory)


In [ ]:
'Some negative values are appearing in the LCS score calculation when mutating from S/Y or T/Y'
#Prevents division by zero 
#Sets min and max values based on that dataset rather than hard-code between 310 and 1.

In [ ]:
import os
import pandas as pd
import re
import plotly.express as px
import numpy as np
from difflib import SequenceMatcher
import kinase_library as kl
import requests 

# Function to extract genomic information from UniProt
def extract_genomic_information_from_uniprot_id(uniprot_id, reference_position):
    try:
        print(f'Searching for UniProt ID: {uniprot_id}')
        requestURL_protein = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprot_id}"
        response_protein = requests.get(requestURL_protein, headers={"Accept": "application/json"})
        response_protein.raise_for_status()
        response_protein = response_protein.json()

        protein_sequence = response_protein.get("sequence", {}).get("sequence", "")
        if not protein_sequence:
            print(f"No sequence found for UniProt ID: {uniprot_id}")
            return None

        reference_position -= 1  # Convert to 0-based index for Python
        start = max(0, reference_position - 7)
        end = min(len(protein_sequence), reference_position + 8)
        extracted_sequence = protein_sequence[start:end]

        return {
            "UniProt_ID": uniprot_id,
            "Protein_Name": response_protein.get("protein", {}).get("recommendedName", {}).get("fullName", ""),
            "TaxID": response_protein.get("organism", {}).get("taxid", ""),
            "Sequence_Length": len(protein_sequence),
            "Reference_Position": reference_position + 1,
            "Extracted_Sequence": extracted_sequence
        }

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to generate all mutated sequences
def generate_mutated_sequences(seq_of_interest):
    mutated_sequences = []
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    for i, original_aa in enumerate(seq_of_interest):
        for new_aa in amino_acids:
            if new_aa != original_aa:
                mutated_seq = seq_of_interest[:i] + new_aa + seq_of_interest[i+1:]
                mutated_sequences.append((i, original_aa, new_aa, mutated_seq))

    return mutated_sequences

# Function to save wild-type predictions
def save_wildtype_predictions(uniprot_id, reference_position, extracted_sequence):
    try:
        wild_type_substrate = kl.Substrate(extracted_sequence)
        wild_type_predictions = wild_type_substrate.predict()
        wild_type_filename = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
        pd.DataFrame(wild_type_predictions).reset_index().to_csv(wild_type_filename, index=False)
        print(f"Wild-type sequence predictions saved to {wild_type_filename}")
    except Exception as e:
        print(f"Error processing wild-type sequence: {e}")

# Function to save mutated sequences
def save_mutated_predictions(mutated_sequences, uniprot_id, reference_position):
    for i, original_aa, new_aa, mutated_seq in mutated_sequences:
        try:
            s = kl.Substrate(mutated_seq)
            predictions = s.predict()
            filename = f"mutations2/{uniprot_id}_{reference_position}_mutation_pos{i+1}_{original_aa}_to_{new_aa}.csv"
            pd.DataFrame(predictions).reset_index().to_csv(filename, index=False)
        except Exception as e:
            print(f"Skipping mutation at position {i+1} ({original_aa} → {new_aa}): {e}")

# Function to extract kinase orders from CSV files
def extract_kinase_orders():
    kinase_orders = {}
    for file in os.listdir("mutations2"):
        if file.endswith(".csv") and "wildtype" not in file:
            file_path = os.path.join("mutations2", file)
            try:
                df = pd.read_csv(file_path)

                # Check for the 'index' column first, then 'position' if 'index' is missing
                if "index" in df.columns:
                    kinase_orders[file] = df["index"].tolist()  # Use 'index' column for kinase order
                elif "Position" in df.columns:
                    kinase_orders[file] = df["Position"].tolist()  # Use 'position' column if 'index' is missing
                else:
                    print(f"Skipping {file}: No 'index' or 'position' column found.")
            except Exception as e:
                print(f"Error reading {file}: {e}")

    return kinase_orders

# Function to compute Longest Common Subsequence (LCS)
def longest_common_subsequence(seq1, seq2):
    matcher = SequenceMatcher(None, seq1, seq2)
    return sum(block.size for block in matcher.get_matching_blocks())

# ✅ Updated Function to normalize LCS scores safely
def normalize_lcs_scores(lcs_scores):
    if not lcs_scores:
        return {}
    
    lcs_min = min(lcs_scores.values())
    lcs_max = max(lcs_scores.values())

    if lcs_max == lcs_min:
        return {file: 1.0 for file in lcs_scores}

    return {file: (score - lcs_min) / (lcs_max - lcs_min) for file, score in lcs_scores.items()}

# Function to process LCS data for heatmap and save it
def process_lcs_for_heatmap(normalized_lcs, uniprot_id, reference_position):
    lcs_data = []
    for file, normalized_score in normalized_lcs.items():
        match = re.match(rf"{uniprot_id}_{reference_position}_mutation_pos(\d+)_(\w+)_to_(\w+)\.csv", file)
        if match:
            position = int(match.group(1))
            mutated_aa = match.group(3)
            lcs_data.append([position, mutated_aa, normalized_score])

    df_lcs = pd.DataFrame(lcs_data, columns=['Position', 'Mutation', 'LCS_Ratio'])
    heatmap_filename = f"mutations2/{uniprot_id}_{reference_position}_heatmap_data.csv"
    df_lcs.to_csv(heatmap_filename, index=False)
    print(f"LCS data saved to {heatmap_filename}")

import plotly.express as px
import numpy as np

# Function to render heatmap from processed data
def render_heatmap_from_data(heatmap_data):
    heatmap_matrix = heatmap_data.pivot_table(index='Mutation', columns='Position', values='LCS_Ratio', aggfunc=np.mean)

    fig = px.imshow(
        heatmap_matrix,
        labels={'x': 'Mutation Position', 'y': 'Mutated Amino Acid', 'color': 'LCS Ratio'},
        color_continuous_scale='Temps',
        title="Heatmap of LCS Ratios for Each Mutation",
    )

    position_labels = list(range(-7, 8))

    fig.update_layout(
        xaxis=dict(tickmode='array', tickvals=list(range(1, 16)), ticktext=position_labels),
        yaxis=dict(tickmode='linear', dtick=1),
        coloraxis_colorbar=dict(
            tickvals=[0, 0.25, 0.5, 0.75, 1],
            ticktext=["0.0", "0.25", "0.5", "0.75", "1.0"],
            tickmode='array',
            title="LCS Ratio",
            tickangle=0
        ),
        plot_bgcolor='white'
    )

    fig.show()


import pandas as pd

# Main pipeline function
def run_pipeline_from_excel(input_file):
    # Load the input Excel file
    df = pd.read_excel(input_file)

    # Iterate over each row in the Excel file
    for index, row in df.iterrows():
        # Extract the UniProt ID and reference position from the row
        uniprot_id = row["uniprot_ID"]
        reference_position = int(row["amino_acid_position"])

        print(f"Processing {uniprot_id} at position {reference_position}...")

        # Extract genomic information
        result = extract_genomic_information_from_uniprot_id(uniprot_id, reference_position)

        if result:
            # Generate mutated sequences
            mutated_sequences = generate_mutated_sequences(result['Extracted_Sequence'])

            # Save wildtype and mutated predictions
            save_wildtype_predictions(uniprot_id, reference_position, result['Extracted_Sequence'])
            save_mutated_predictions(mutated_sequences, uniprot_id, reference_position)

            # Extract kinase orders
            kinase_orders = extract_kinase_orders()

            # Load the wildtype CSV file
            wildtype_path = f"mutations2/{uniprot_id}_{reference_position}_wildtype.csv"
            try:
                df_wildtype = pd.read_csv(wildtype_path)

                # Check if either "index" or "position" columns are present
                if "index" not in df_wildtype.columns and "position" not in df_wildtype.columns:
                    print(f"Error: Neither 'index' nor 'position' column found in {wildtype_path}")
                    continue  # Skip this entry and go to the next one

                # Use 'index' or 'position' for kinase order
                kinase_order_column = "index" if "index" in df_wildtype.columns else "position"
                wildtype_kinase_order = df_wildtype[kinase_order_column].tolist()

                # Calculate LCS scores
                lcs_scores = {file: longest_common_subsequence(order, wildtype_kinase_order) for file, order in kinase_orders.items()}

                # Normalize LCS scores
                normalized_lcs = normalize_lcs_scores(lcs_scores)

                # Process LCS for heatmap and save it
                process_lcs_for_heatmap(normalized_lcs, uniprot_id, reference_position)
                render_heatmap_from_data(pd.read_csv(f"mutations2/{uniprot_id}_{reference_position}_heatmap_data.csv"))

            except FileNotFoundError:
                print(f"Warning: Wildtype file {wildtype_path} not found. Skipping {uniprot_id}.")
                continue

        else:
            print(f"Error: No genomic information found for {uniprot_id} at position {reference_position}. Skipping.")
            continue


if __name__ == "__main__":
    run_pipeline_from_excel("input.xlsx")


In [ ]:
'Compare benign and pathogenic mutation LCS scores across mutation and position' 
